## Let see what's the content of the file

In [1]:
i=0
with open("./data/-") as infile:
    for line in infile:
        print(line)
        i = i+1
        if i == 15:
            break

Your mission shall you accept it is to

* Explain every column.

* Find the 3 richest person.



This is a simple test, it should not take you more 30minutes to code.







a,b,c,d,e,f,g

0,longue description 0,Victor,Évrat,1779,0,1779

1,longue description 1,Ugo,Ogier,3218,0,3218

2,longue description 2,Gerard,Zola,3909,0,3909

3,longue description 3,Ingrid,Quesada,2672,0,2672

4,longue description 4,Xavier,Yvars,4224,0,4224

5,longue description 5,Régis,Ferry,3479,0,3479



#### The file contains a text and the data. 

#### Now, we'll delete the text and keep only the data

In [1]:
input_file_path = './data/-'
output_file_path = 'output.txt'

with open(input_file_path, 'r') as infile, open(output_file_path, 'w') as outfile:
    # Skip the first 9 lines
    for _ in range(9):
        next(infile)

    # Write the remaining lines to the output file
    for line in infile:
        outfile.write(line)

print(f'Lines removed. Output file saved at: {output_file_path}')


Lines removed. Output file saved at: output.txt


### We have now a new text file with only data



#### We rename manually the *output.txt* to *output.csv*

In [6]:
import dask.dataframe as dd

file_path = 'output.csv'

ddf = dd.read_csv(file_path) 
ddf.columns = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
ddf.head()

,a,b,c,d,e,f,g
0,1,longue description 1,Ugo,Ogier,3218,0,3218
1,2,longue description 2,Gerard,Zola,3909,0,3909
2,3,longue description 3,Ingrid,Quesada,2672,0,2672
3,4,longue description 4,Xavier,Yvars,4224,0,4224
4,5,longue description 5,Régis,Ferry,3479,0,3479


## Let explain each column
* column *a* : Index or ID of each row
* column *b* : Detailed description or long description associated with each entry.
* column *c* : First name of the person associated with each entry.
* column *d* : Last name of the person associated with each entry.
* column *e* : Weekly or monthly income
* column *f* : Weekly or monthly debt
* column *g* : total wealth

### Let keep and rename necessary columns

In [7]:
ddf['full name'] = ddf['d'] + ' ' + ddf['c']
ddf = ddf.rename(columns={"e":"income", "f": "debt", "g": "wealth"})

ddf = ddf[["full name", "income", "debt", "wealth"]]
ddf.head()

,full name,income,debt,wealth
0,Ogier Ugo,3218,0,3218
1,Zola Gerard,3909,0,3909
2,Quesada Ingrid,2672,0,2672
3,Yvars Xavier,4224,0,4224
4,Ferry Régis,3479,0,3479


In [11]:
ddf.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 4 entries, full name to wealth
dtypes: object(1), int64(3)

#### We divide our dataset into 20 partitions with *Dask*, as the data is very large



In [12]:
ddf_chunks = ddf.repartition(npartitions=20)

#### Let assume first that there is no duplicate name in our data, that mean, each person appear just one and display the richest person of each partition

In [14]:
n_largest = 3

# let's display the 3 richest person of each partition
result = ddf_chunks.map_partitions(lambda df: df.nlargest(n_largest, 'wealth')).compute()

print(result)

              full name  income  debt     wealth
1052691  Yvars Béatrice     267     0   45426698
1052659  Yvars Béatrice     143     0   45426431
1052002  Yvars Béatrice    4785     0   45426288
1052675       Évrat Zoé    3976     0   92796382
1052127       Évrat Zoé     327     0   92792406
1051732       Évrat Zoé    4961     0   92792079
1035754    Aubry Pascal    3615     0  136213069
1035303    Aubry Pascal    3070     0  136209598
1035553    Aubry Pascal       0  -144  136209454
1033872   Geneau Océane     901     0  182419762
1033654   Geneau Océane    3714     0  182418861
1033038   Geneau Océane    4026     0  182415147
1035457   Xharde Xavier    3968     0  229253557
1035553   Xharde Xavier       0  -114  229253443
1034750   Xharde Xavier    2661     0  229249589
1035377   Xharde Xavier     462     0  272584780
1035222   Xharde Xavier    4894     0  272584318
1034792   Xharde Xavier    2681     0  272579685
1019125   Xharde Xavier     797     0  318185048
1018578   Xharde Xav

## We can notice that a person appears several times with different income, debt, and wealth.

Let's look deeper in "Yvars Béatrice" records for exemple

In [16]:
first_chunk = ddf_chunks.get_partition(0).compute()
desired_rows = first_chunk[first_chunk['full name'] == 'Yvars Béatrice']

print(desired_rows)

              full name  income  debt    wealth
2284     Yvars Béatrice      24     0        24
3359     Yvars Béatrice    1603     0      1627
3736     Yvars Béatrice    2915     0      4542
7209     Yvars Béatrice    2381     0      6923
8509     Yvars Béatrice       0  -777      6146
...                 ...     ...   ...       ...
1049692  Yvars Béatrice    4530     0  45417572
1051104  Yvars Béatrice    3931     0  45421503
1052002  Yvars Béatrice    4785     0  45426288
1052659  Yvars Béatrice     143     0  45426431
1052691  Yvars Béatrice     267     0  45426698

[22561 rows x 4 columns]


We can observe an interesting pattern: the name "Yvars Béatrice" appears 22,561 times.

 Upon closer examination, we can notice that in the first occurrence, the "income" amount equals the "wealth," and the "debt" is zero, which appears logically consistent. Subsequent rows represent the sum of previous "wealth" and the new "income." minus the "debt" Therefore, the last recorded line corresponds to the total wealth accumulated by "Yvars Béatrice."

#### In summary, this repetition of the name with variations in the "income," "wealth," and "debt" columns seems to follow a logical pattern, where the last line reflects the total wealth of "Yvars Béatrice", and that is all we need

## Next step: Eliminate all duplicates based on the "full name" and retain only the latest occurrence.

In [17]:
# Remove duplicates by keeping the last occurrence in each chunk
ddf_no_duplicates = ddf_chunks.map_partitions(lambda df: df.drop_duplicates(subset=['full name'], keep='last')).compute()


In [18]:
ddf_no_duplicates.shape

(13520, 4)

In [19]:
ddf_no_duplicates = ddf_no_duplicates.sort_values("wealth",ascending=False)

In [20]:
ddf_no_duplicates.head(3)

,full name,income,debt,wealth
1018905,Walliand Zoé,0,-65,893775893
1018982,Parmentier Gerard,0,-487,892220076
1019360,Urbain Océane,1960,0,891910460


## The 3 richest person:

* Walliand Zoé : 893775893
* Parmentier Gerard : 892220076
* Urbain Océane : 891910460